# Introduction
The goal of this script is to demonstrate dbt Cloud REST API capabilities, for running and leveraging Jobs and their metadata.

Disclaimer :
This is free and unencumbered software released into the public domain.

Anyone is free to copy, modify, publish, use, compile, sell, or
distribute this software, either in source code form or as a compiled
binary, for any purpose, commercial or non-commercial, and by any
means.

In jurisdictions that recognize copyright laws, the author or authors
of this software dedicate any and all copyright interest in the
software to the public domain. We make this dedication for the benefit
of the public at large and to the detriment of our heirs and
successors. We intend this dedication to be an overt act of
relinquishment in perpetuity of all present and future rights to this
software under copyright law.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND,
EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF
MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT.
IN NO EVENT SHALL THE AUTHORS BE LIABLE FOR ANY CLAIM, DAMAGES OR
OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE,
ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR
OTHER DEALINGS IN THE SOFTWARE.

For more information, please refer to <https://unlicense.org>

### 1. Importing Librairies

In [36]:
import requests
import json
import time
from getpass import getpass

### 2. Initiating Connection to the dbt Cloud using the API.

The following two cells define the API token and account/project IDs for both source and target dbt Cloud environments. These credentials are essential for authentication and specifying which environments to work with. For security best practices, consider using environment variables or secure vaults to handle credentials.




In [37]:
# Define parameters needed for API access on the Source - ACCOUNT YOU ARE MIGRATING FROM
dbt_api_token = getpass('Source API TOKEN:') # don't edit this, enter when prompter
dbt_account_id = 70403103916174 #
dbt_project_id = 70403103944286 # We might need it
dbt_environment_id = 70403103945940 # We would not need it
dbt_base_url = 'https://c1.us1.dbt.com'

Source API TOKEN:··········


### 3. Functions and helpers

In [61]:
#from typing_extensions import NamedTuple
# Helper function to set headers
def set_headers(token):
    return {
        'Authorization': f'Token {token}',
        'Content-Type': 'application/json'
    }

# 1) Run a dbt Job given a Job ID, return run_id
def run_dbt_job(job_id):
    url = f"{dbt_base_url}/api/v2/accounts/{dbt_account_id}/jobs/{job_id}/run/"
    headers = set_headers(dbt_api_token)
    payload = {
        "cause": "Triggered via API"
    }
    response = requests.post(url, headers=headers, json=payload)
    if response.status_code == 200:
        run_id = response.json()['data']['id']
        return run_id
    else:
        response.raise_for_status()

# 2) a) Re-run a dbt Job using the API
def rerun_dbt_job(job_id):
    url = f"{dbt_base_url}/api/v2/accounts/{dbt_account_id}/jobs/{job_id}/rerun/"
    headers = set_headers(dbt_api_token)
    payload = {
        "cause": "Re-triggered via API"
    }
    response = requests.post(url, headers=headers, json=payload)
    if response.status_code == 200:
        print("Job re-triggered successfully.")
    else:
        response.raise_for_status()

# 3) Given a run_id, return results_run.json and manifest.json
def get_run_results(run_id):
    url = f"{dbt_base_url}/api/v2/accounts/{dbt_account_id}/runs/{run_id}/artifacts/"
    headers = set_headers(dbt_api_token)

    # Fetching results_run.json
    results_url = f"{url}run_results.json"
    results_response = requests.get(results_url, headers=headers)
    if results_response.status_code == 200:
        results_run = results_response.json()
    else:
        results_response.raise_for_status()

    # Fetching manifest.json
    manifest_url = f"{url}manifest.json"
    manifest_response = requests.get(manifest_url, headers=headers)
    if manifest_response.status_code == 200:
        manifest = manifest_response.json()
    else:
        manifest_response.raise_for_status()

    return results_run, manifest

# 4) Setup a new Job in project and environment defined, define the commands and return jobs_id
def setup_new_job(job_data):
    url = f"{dbt_base_url}/api/v2/accounts/{dbt_account_id}/jobs/"
    headers = set_headers(dbt_api_token)

    # Debugging information
    #print("URL:", url)
    #print("Headers:", headers)
    #print("Payload:", job_data)

    response = requests.post(url, headers=headers, json=job_data)

    # Debugging information
    #print("Response status code:", response.status_code)
    #print("Response content:", response.content)

    if response.status_code == 201:
        job_id = response.json()['data']['id']
        return job_id
    else:
        print("Error details:", response.json())
        response.raise_for_status()


### 4. Example usage of functions

#### 4.1 Run a dbt Cloud Job

In [ ]:
job_id = 70403103960876  # Example job ID
run_id = run_dbt_job(job_id)
print(f"Run ID: {run_id}")


#### 4.2 Re-run a dbt Job

Use this endpoint to retry a failed run for a job from the point of failure, if the run failed. Otherwise trigger a new run. When this endpoint returns a successful response, a new run will be enqueued for the account. Users can poll the Get run endpoint to poll the run until it completes. After the run has completed, users can use the Get run artifact endpoint to download artifacts generated by the run.


In [ ]:
job_id = 70403103960876
rerun_dbt_job(job_id)

#### 4.3 Get Manifest Data from a Given Run

Get Metadata from a given run.


In [ ]:
run_id = 70403116972055
results, manifest = get_run_results(run_id)
print("Results:", results)
print("Manifest:", manifest)

#### 4.4 Create a Job with Custom Commands

Create a new job with Custom dbt Commands


In [62]:
# Defining Job Inputs :
job_data = {
    "id": None,
    "name": "New Example Job",
    "execute_steps": ["dbt source freshness", "dbt build --select +stg_tpch_nations"],
    "dbt_version": None,  # Example version, adjust as needed
    "environment_id": dbt_environment_id,
    "project_id": dbt_project_id,
    "account_id": dbt_account_id,
    "state": 1,
    "triggers": {
        "github_webhook": False,
        "git_provider_webhook": False,
        "schedule": False,
        "custom_branch_only": False
    },
    "settings": {
        "threads": 4,
        "target_name": "default"
    },
    "schedule": None
}

In [ ]:
try:
    new_job_id = setup_new_job(job_data)
    print(f"New Job ID: {new_job_id}")
except requests.exceptions.HTTPError as err:
    print("HTTP error occurred:", err)
except Exception as err:
    print("An error occurred:", err)